In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import networkx as nx
import plotly.offline as py
from collections import Counter

In [176]:
N_link = 10 # N is the number of relations of a character (enables us to delete low relationship)

### Load data & Convert to graph format 

In [184]:
df_surronding_PER = pd.read_csv(r'res_surronding_PER/1869_Flaubert-Gustave_L-Education-sentimentale.csv') # df surronding PER

/tmp/ipykernel_51726/3578804121.py:1: DtypeWarning:

Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.



In [185]:
df_surronding_PER

,<Frédéric>,<Arnoux>,<Deslauriers>,<Mme Arnoux>,<Rosanette>,<Hussonnet>,<Pellerin>,<!Frédéric>,<Sénécal>,<Ils>,<ils>,<M . Dambreuse>,<Regimbart>,<elle>,<!Il>,<Martinon>,<Mlle Roque>,<on>,<la Maréchale>
0,<plusieurs>,<une jeune dame et un jeune homme>,<Deslauriers qui>,<ils>,<Mme Rosanette l'>,<on>,<Les confrères absents>,<Deslauriers>,<Sylvio>,<Frédéric>,<Frédéric>,<son mari>,<Arnoux>,<Un jeune homme de dix - huit ans>,<Ils>,"<Ségur , Dupeyron et Mauroy>",<Deslauriers>,<Frédéric>,"<les dames , posées au bord des chaises>"
1,"<trois ébénistes , en blouse>",<un jeune homme>,<Deslauriers qui>,<ils>,<Mme Rosanette l'>,<on>,<On>,<M . Dambreuse>,<ils>,<Frédéric>,<Frédéric>,<Mme Moreau>,<Arnoux>,<Un jeune homme de dix - huit ans>,<Ils>,<Dupeyron et Mauroy>,<Deslauriers>,<Frédéric>,"<les dames , posées au bord des chaises>"
2,<un joueur de harpe en haillons>,<La petite fille>,<du Christ>,<ils>,<Mme Rosanette l'>,<on>,<Les confrères absents>,<Rastignac>,<ils>,<Frédéric>,<Frédéric>,<M . Gamblin lui>,<Arnoux>,<Un jeune homme de dix - huit ans>,<Ils>,<Mauroy>,<Deslauriers>,<Frédéric>,<Frédéric l'>
3,<un joueur de harpe en haillons>,<lui>,<Deslauriers qui>,<Mme Arnoux>,<Frédéric>,<un petit jeune homme qui>,<tous>,<M . Dambreuse>,<ils>,<Frédéric>,<Frédéric>,<M . Gamblin lui>,<Arnoux>,<M . Frédéric Moreau>,<ils>,<ils>,<Deslauriers>,<Frédéric>,<Mme Dambreuse les>
4,<le capitaine>,<Frédéric>,"<Froissart , Commines , Pierre de l' Estoile>",<ils>,<Frédéric>,<on>,<un homme de taille moyenne>,<Deslauriers>,<ils>,<une petite fille>,<ils>,<Mme Moreau>,<Arnoux>,<Un jeune homme de dix - huit ans>,<ils>,<Buloz>,<Deslauriers>,<ils>,<Frédéric l'>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40285,<Rosanette>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40286,<Rosanette>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40287,<Mme Arnoux>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40288,<Mme Arnoux>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [186]:
# network as get dict of dict

In [187]:
def get_dict_of_dict(df_res, N_link):
    dict_res = {}
    for column_perso in df_res.columns:
        dict_res[column_perso] = dict(Counter(df_res[column_perso]).most_common(N))
    return dict_res

In [188]:
dict_surronding_PER = get_dict_of_dict(df_surronding_PER.dropna(), N)

In [189]:
dict_surronding_PER

{'<Frédéric>': {'<Frédéric>': 1871,
  '<une petite fille>': 359,
  '<:Elle>': 196,
  '<Le Capitaine>': 176,
  '<Ils>': 83,
  '<leurs>': 58,
  '<La petite fille>': 56,
  "<Une négresse , coiffée d' un foulard>": 55,
  "<Deslauriers l'>": 46,
  '<Arnoux>': 37},
 '<Arnoux>': {'<Arnoux>': 1393,
  '<Frédéric>': 182,
  '<du NationalLe National>': 137,
  '<Mlle Marthe>': 117,
  "<Arnoux l'>": 97,
  '<un homme de taille moyenne>': 70,
  "<l' illustre Pierre - Paul Meinsius>": 65,
  '<La petite fille>': 58,
  '<Hussonnet>': 57,
  '<lui>': 55},
 '<Deslauriers>': {'<Deslauriers>': 1709,
  '<M . Dambreuse>': 249,
  "<l'>": 193,
  '<!Frédéric>': 99,
  '<le tailleur>': 96,
  '<Mme Arnoux>': 84,
  '<notre>': 76,
  '<ils>': 72,
  '<les Dambreuse>': 64,
  "<Arnoux l'>": 56},
 '<Mme Arnoux>': {'<Mme Arnoux>': 1506,
  "<l'>": 313,
  "<Arnoux l'>": 297,
  '<!Frédéric>': 237,
  '<Martinon>': 163,
  '<Deslauriers>': 87,
  '<au juif Isaac>': 53,
  '<ces hommes>': 49,
  '<Pellerin>': 45,
  '<Rosenwald>': 37},

In [190]:
# get weights with mention counter

In [191]:
dict_PER = pd.read_csv(r'res_mention_PER/1869_Flaubert-Gustave_L-Education-sentimentale.csv') # df mention PER (Counter)

In [192]:
dict_PER_mention = dict(zip(list(dict_PER.mention), list(dict_PER['count'])))

In [193]:
dict_PER_mention

{'<Frédéric>': 1465,
 '<Arnoux>': 1126,
 '<Deslauriers>': 634,
 '<Mme Arnoux>': 629,
 '<Rosanette>': 570,
 '<Hussonnet>': 469,
 '<Pellerin>': 446,
 '<!Frédéric>': 360,
 '<Sénécal>': 329,
 '<Ils>': 319,
 '<ils>': 318,
 nan: 303,
 '<M . Dambreuse>': 292,
 '<Regimbart>': 278,
 '<elle>': 236,
 '<!Il>': 226,
 '<Martinon>': 220,
 '<Mlle Roque>': 201,
 '<on>': 161,
 '<la Maréchale>': 161}

### Visualisation 

In [194]:
GraphEducation = nx.Graph()

In [195]:
# Add node for each character
for perso in dict_PER_mention.keys():
    GraphEducation.add_node(perso, size = dict_PER_mention[perso])

In [196]:
GraphEducation.nodes

NodeView(('<Frédéric>', '<Arnoux>', '<Deslauriers>', '<Mme Arnoux>', '<Rosanette>', '<Hussonnet>', '<Pellerin>', '<!Frédéric>', '<Sénécal>', '<Ils>', '<ils>', nan, '<M . Dambreuse>', '<Regimbart>', '<elle>', '<!Il>', '<Martinon>', '<Mlle Roque>', '<on>', '<la Maréchale>'))

In [197]:
# For each co-appearance between two characters, add an edge
for char in dict_PER_mention.keys():
    for co_char in dict_surronding_PER[char].keys():
        
        # Only add edge if mention surronding is in dict_PER_mention.keys()
        
        if co_char in dict_PER_mention.keys():
            
        # Only add edge if the count is positive
            if dict_surronding_PER[char][co_char] > 0:
        # Only add an edge if char != co_char
                if char != co_char:
                    GraphEducation.add_edge(char, co_char, weight = dict_surronding_PER[char][co_char])

KeyError: nan

In [169]:
dict_surronding_PER

{'<Frédéric>': {'<Frédéric>': 1065,
  '<une petite fille>': 184,
  '<:Elle>': 87,
  '<Le Capitaine>': 83,
  '<La petite fille>': 37,
  '<Ils>': 36,
  "<Une négresse , coiffée d' un foulard>": 33,
  '<M . le censeur>': 26,
  '<leurs>': 21,
  '<deux chasseurs>': 15},
 '<Arnoux>': {'<Arnoux>': 883,
  '<du NationalLe National>': 66,
  '<Mlle Marthe>': 62,
  '<Frédéric>': 58,
  "<Arnoux l'>": 53,
  "<l' illustre Pierre - Paul Meinsius>": 40,
  '<Mlle Vatnaz>': 36,
  '<lui>': 27,
  '<le vieux cocher>': 27,
  '<une jeune fille blonde>': 27},
 '<Deslauriers>': {'<Deslauriers>': 971,
  '<M . Dambreuse>': 127,
  "<l'>": 84,
  '<!Frédéric>': 46,
  '<notre>': 42,
  '<ils>': 41,
  '<les Dambreuse>': 31,
  '<Mme Moreau>': 30,
  '<Mme Arnoux>': 30,
  '<le tailleur>': 27},
 '<Mme Arnoux>': {'<Mme Arnoux>': 906,
  "<Arnoux l'>": 159,
  "<l'>": 148,
  '<!Frédéric>': 121,
  '<Martinon>': 73,
  '<Deslauriers>': 32,
  '<au juif Isaac>': 23,
  '<ces hommes>': 23,
  '<Pellerin>': 20,
  '<Rosenwald>': 17},
 '

In [170]:
GraphEducation.edges()

EdgeView([('<Frédéric>', '<Ils>'), ('<Frédéric>', '<Arnoux>'), ('<Frédéric>', '<Rosanette>'), ('<Frédéric>', '<Pellerin>'), ('<Frédéric>', '<!Frédéric>'), ('<Frédéric>', '<Regimbart>'), ('<Frédéric>', '<on>'), ('<Frédéric>', '<la Maréchale>'), ('<Arnoux>', '<Rosanette>'), ('<Arnoux>', '<M . Dambreuse>'), ('<Arnoux>', '<Regimbart>'), ('<Arnoux>', '<elle>'), ('<Arnoux>', '<on>'), ('<Arnoux>', '<la Maréchale>'), ('<Deslauriers>', '<M . Dambreuse>'), ('<Deslauriers>', '<!Frédéric>'), ('<Deslauriers>', '<ils>'), ('<Deslauriers>', '<Mme Arnoux>'), ('<Deslauriers>', '<Sénécal>'), ('<Deslauriers>', '<Regimbart>'), ('<Deslauriers>', '<Martinon>'), ('<Mme Arnoux>', '<!Frédéric>'), ('<Mme Arnoux>', '<Martinon>'), ('<Mme Arnoux>', '<Pellerin>'), ('<Mme Arnoux>', '<Sénécal>'), ('<Mme Arnoux>', '<Ils>'), ('<Mme Arnoux>', '<elle>'), ('<Mme Arnoux>', '<!Il>'), ('<Mme Arnoux>', '<la Maréchale>'), ('<Rosanette>', '<Hussonnet>'), ('<Rosanette>', '<Pellerin>'), ('<Rosanette>', '<on>'), ('<Rosanette>', '<I

In [171]:
# Get positions for the nodes in G
pos_ = nx.spring_layout(GraphEducation)

In [172]:
def make_edge(x, y, text, width):
    
    '''Creates a scatter trace for the edge between x's and y's with given width

    Parameters
    ----------
    x    : a tuple of the endpoints' x-coordinates in the form, tuple([x0, x1, None])
    
    y    : a tuple of the endpoints' y-coordinates in the form, tuple([y0, y1, None])
    
    width: the width of the line

    Returns
    -------
    An edge trace that goes between x0 and x1 with specified width.
    '''
    return  go.Scatter(x         = x,
                       y         = y,
                       line      = dict(width = width,
                                   color = 'cornflowerblue'),
                       hoverinfo = 'text',
                       text      = ([text]),
                       mode      = 'lines')

In [173]:
# For each edge, make an edge_trace, append to list
edge_trace = []
for edge in GraphEducation.edges():
    
    if GraphEducation.edges()[edge]['weight'] > 0:
        char_1 = edge[0]
        char_2 = edge[1]

        x0, y0 = pos_[char_1]
        x1, y1 = pos_[char_2]

        text   = char_1 + '--' + char_2 + ': ' + str(GraphEducation.edges()[edge]['weight'])
        
        trace  = make_edge([x0, x1, None], [y0, y1, None], text,
                           GraphEducation.edges()[edge]['weight']/5)

        edge_trace.append(trace)

In [174]:
# Make a node trace
node_trace = go.Scatter(x         = [],
                        y         = [],
                        text      = [],
                        textposition = "top center",
                        textfont_size = 30,
                        mode      = 'markers+text',
                        hoverinfo = 'none',
                        marker    = dict(color = [],
                                         size  = [],
                                         line  = None))
# For each node in midsummer, get the position and size and add to the node_trace
for node in GraphEducation.nodes():
    x, y = pos_[node]
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])
    node_trace['marker']['color'] += tuple(['cornflowerblue'])
    node_trace['marker']['size'] += tuple([GraphEducation.nodes()[node]['size']/10])
    node_trace['text'] += tuple(['<b>' + node + '</b>'])

In [175]:
layout = go.Layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)


fig = go.Figure(layout = layout)

for trace in edge_trace:
    fig.add_trace(trace)

fig.add_trace(node_trace)

fig.update_layout(showlegend = False)

fig.update_xaxes(showticklabels = False)

fig.update_yaxes(showticklabels = False)

fig.show()
py.plot(fig, filename='GraphEducation.html')

'GraphEducation.html'

In [121]:
#AVEC N PER = 20 et N_token = 50, N_link = 10
layout = go.Layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)


fig = go.Figure(layout = layout)

for trace in edge_trace:
    fig.add_trace(trace)

fig.add_trace(node_trace)

fig.update_layout(showlegend = False)

fig.update_xaxes(showticklabels = False)

fig.update_yaxes(showticklabels = False)

fig.show()
py.plot(fig, filename='GraphEducation.html')

'GraphEducation.html'